In [1]:
# Loading the library to python 
import sys
sys.path.insert(0, "anomaly_detection/")

In [2]:
import pandas as pd
import os
import numpy as np
import random

# Reading data

In [3]:
def pick_feats(n, w = None, p = None, path_feats = "features"):
    df = pd.DataFrame()
    n_net = 0
    for file in os.listdir(path_feats):
        if file[0] != "G" and (n is None or "n{}_".format(n) in file)\
                          and (w is None or "w{}_".format(w) in file)\
                          and (p is None or "p{}_".format(p) in file):
            df_net = pd.read_pickle(os.path.join(path_feats, file))
            df_net["net_id"] = n_net
            df = df.append(df_net, ignore_index = True)
            n_net += 1
    return df

In [4]:
def split_net(df, test_prop = 0.2):
    net_ids = df.net_id.unique()
    n_net = len(net_ids) 
    test_net = np.random.choice(net_ids, int(max(round((test_prop)*n_net), 1)))
    selector = df.net_id.isin(test_net)
    df_train = df.loc[~selector]
    df_test = df.loc[selector]
    return df_train, df_test

In [23]:
df = pick_feats(n = 500)
df = df.replace([np.inf, -np.inf, np.nan], 0)
df.shape

(11500, 90)

In [24]:
df_train, df_test = split_net(df, test_prop=0.2)
df_test.shape, df_train.shape

((2500, 90), (9000, 90))

# Fitting

In [25]:
from sklearn.ensemble import RandomForestClassifier

In [26]:
clf = RandomForestClassifier(n_estimators=100)

In [27]:
clf.fit(df_train.drop("anomaly", axis = 1), df_train.anomaly)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [28]:
Y_test_pred = clf.predict(df_test.drop("anomaly", axis = 1))

In [29]:
Y_test = df_test.anomaly

In [30]:
from sklearn.metrics import precision_score, recall_score

In [31]:
precision_score(Y_test, Y_test_pred), recall_score(Y_test, Y_test_pred)

(0.8363636363636363, 0.2391681109185442)